In [1]:
import torch 
import torch.nn as nn 
import numpy as np 

x1 = np.array([0,2,4,6,8,0,2,2,2,2,2,6,8,]) 
x2 = np.array([1,3,1,5,7,9,7]) 
x3 = np.array([5,3,1,7,3]) 
x4 = np.array([7,3,9,1,9,9,9,9,9,7]) 
x5 = np.array([8,6,4,2,4,4,2,0,8]) 
x6 = np.array([8,4,0,0,0,0,0,2]) 

x = list() 
x.append(x1) 
x.append(x2) 
x.append(x3) 
x.append(x4) 
x.append(x5) 
x.append(x6)

y1 = np.array([1.0]) 
y2 = np.array([0.0]) 
y3 = np.array([0.0]) 
y4 = np.array([0.0]) 
y5 = np.array([1.0]) 
y6 = np.array([1.0]) 

y = list() 
y.append(y1) 
y.append(y2) 
y.append(y3) 
y.append(y4) 
y.append(y5) 
y.append(y6)


In [2]:
def biner(index):
    buffer  = np.zeros([1,10])
    buffer[0][index]=1.0
    return buffer

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.in2output = nn.Linear(input_size + hidden_size, output_size)
    
    def forward(self, x, hidden_state):
        combined = torch.cat((x, hidden_state), 1)
        hidden = torch.sigmoid(self.in2hidden(combined))
        output = self.in2output(combined)
        return output, hidden
    
    def init_hidden(self):
        return nn.init.kaiming_uniform_(torch.empty(1, self.hidden_size))

In [8]:
num_letters = 10 
num_targets = 2 
hidden_size = 15

learning_rate = 0.001
model = RNN(num_letters, hidden_size, num_targets)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(0,1000):
    for i in range(0,len(x)):
        record = x[i] 
        label = torch.tensor(y[i],dtype=torch.int64)
        hidden_state = model.init_hidden()
        for j in range(0,len(record)):        
            vector = biner(record[j])
            inputs = torch.tensor(vector,dtype=torch.float)
            output, hidden_state = model(inputs, hidden_state)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        _, pred = torch.max(output, dim=1)
        print("epoch : ",epoch,"record :",i," target: ",label," pred: ",pred," loss: ",loss.item())

epoch :  0 record : 0  target:  tensor([1])  pred:  tensor([0])  loss:  0.9725450873374939
epoch :  0 record : 1  target:  tensor([0])  pred:  tensor([0])  loss:  0.503324568271637
epoch :  0 record : 2  target:  tensor([0])  pred:  tensor([0])  loss:  0.5971161723136902
epoch :  0 record : 3  target:  tensor([0])  pred:  tensor([0])  loss:  0.4981144666671753
epoch :  0 record : 4  target:  tensor([1])  pred:  tensor([0])  loss:  0.9719855189323425
epoch :  0 record : 5  target:  tensor([1])  pred:  tensor([0])  loss:  0.8722018599510193
epoch :  1 record : 0  target:  tensor([1])  pred:  tensor([0])  loss:  0.9687913656234741
epoch :  1 record : 1  target:  tensor([0])  pred:  tensor([0])  loss:  0.49158307909965515
epoch :  1 record : 2  target:  tensor([0])  pred:  tensor([0])  loss:  0.5858542323112488
epoch :  1 record : 3  target:  tensor([0])  pred:  tensor([0])  loss:  0.4895341694355011
epoch :  1 record : 4  target:  tensor([1])  pred:  tensor([0])  loss:  0.9702866077423096

In [9]:
def predict(record):
    model.eval()
    with torch.no_grad():
        hidden_state = model.init_hidden()
        for j in range(0,len(record)):        
            vector = biner(record[j])
            ##lakukan perhitungan
            inputs = torch.tensor(vector,dtype=torch.float)
            output, hidden_state = model(inputs, hidden_state)
        _, pred = torch.max(output, dim=1)
    model.train()
    return pred

x1 = np.array([8,8,8,8,4,2,8])
print(predict(x1))

tensor([1])
